# Visualizing a 4D Data Cube with Astrowidgets

This is an example of reading and visualizing a four dimensional data cube using Astrowidgets.

In [ ]:
from astropy.nddata import NDData
from astropy.wcs import WCS
from ginga.misc.log import get_logger
from matplotlib import pyplot as plt

import ipywidgets as ipyw
from IPython.display import display, clear_output

from astrowidgets import ImageWidget

%matplotlib inline

# To prevent automatic figure display when execution of the cell ends
# https://github.com/jupyter-widgets/ipywidgets/issues/1940
%config InlineBackend.close_figures=False 

plt.ioff()

Subclass `ImageWidget` to implement cube-specific behavior.

In [ ]:
class CubeWidget(ImageWidget):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self._4d_idx = 0  # Lock 4th dim to this for now
        
        # For line profile plot
        self._cur_islice = None
        self._cur_ix = None
        self._cur_iy = None
        self.line_out = ipyw.Output()
        self.line_plot = None
        self.plot_xlabel = 'Ramp'
        self.plot_ylabel = 'Pixel value'

    def load_nddata(self, nddata, naxispath=None):
        from ginga.AstroImage import AstroImage
        image = AstroImage()
        image.load_nddata(nddata, naxispath=[0, self._4d_idx])
        self._viewer.set_image(image)
        
    def _mouse_click_cb(self, viewer, event, data_x, data_y):
        self._cur_ix = int(round(data_x))
        self._cur_iy = int(round(data_y))
        self.plot_line_profile()
        
        # Ensure only active marker is shown
        self.reset_markers()
        
        super()._mouse_click_cb(viewer, event, data_x, data_y)

    def plot_line_profile(self):
        if self.line_plot is None or self._cur_ix is None or self._cur_iy is None:
            return

        image = self._viewer.get_image()
        if image is None:
            return

        with self.line_out:
            mddata = image.get_mddata()
            self.line_plot.clear()
            self.line_plot.plot(mddata[self._4d_idx, :, self._cur_iy, self._cur_ix], 'b-')

            if self._cur_islice is not None:
                y = mddata[self._4d_idx, self._cur_islice, self._cur_iy, self._cur_ix]
                self.line_plot.plot(self._cur_islice, y, 'ro')

            self.line_plot.set_title(f'X={self._cur_ix + 1} Y={self._cur_iy + 1}')
            self.line_plot.set_xlabel(self.plot_xlabel)
            self.line_plot.set_ylabel(self.plot_ylabel)

            clear_output(wait=True)
            display(self.line_plot.figure)

    def show_slice(self, n):
        image = self._viewer.get_image()
        image.set_naxispath([n, self._4d_idx])
        self._viewer.redraw(whence=0)
        self._cur_islice = n

In [ ]:
logger = get_logger('my viewer', log_stderr=True, log_file=None, level=30)

In [ ]:
w = CubeWidget(logger=logger)

Generate a fake 4D data cube.

**--- OR ---**

Load an existing data cube.

Dask support requires https://github.com/ejeschke/ginga/pull/805.

In [ ]:
import dask.array as da
from astropy.io import fits

# This is a large cube that is 6GB.
filename = '/redkeep/ironthrone/ssb/stginga/test_data/NRCN815A_LIN_20160115_uncal.fits'
pf = fits.open(filename, memmap=True)

# There is no WCS assigned for UNCAL file yet.
# This is a dummy WCS.
im_wcs = WCS(pf[1].header)

# Convert to NDData that wraps a Dask array.
# Must provide Dask with a name to avoid hashing.
# Shape is (1, 190, 2048, 2048)
im = NDData(da.from_array(pf[1].data, name='nircam_sci'), wcs=im_wcs)

Grab the `z`-dimension for slider.

In [ ]:
n_3d = im.data.shape[1]

Load `NDData` into viewer widget.

In [ ]:
# Loading a fake WCS into widget sometimes gives error.
# Try reloading before giving up.
w.load_nddata(im)

Create a slider for third dimension.

In [ ]:
def show_slice(n):
    w.show_slice(n - 1)
    w.plot_line_profile()  # Update the red dot

slider = ipyw.interactive(
    show_slice,
    n=ipyw.IntSlider(min=1, max=n_3d, step=1, value=0, continuous_update=False))

Set up the plot and hook to widget.

In [ ]:
# If plot shows, rerun cell to hide it.
ax = plt.gca();

w.line_plot = ax

This marks the spot being clicked on the viewer.

NOTE: `w.get_markers(marker_name=marker_name)` might not work if the given WCS cannot be successfully converted to `SkyCoord`.

In [ ]:
marker_name = 'line_profile'

# Clean previous call in same session.
w.stop_marking()

w.start_marking(
    marker_name=marker_name,
    marker={'type': 'circle', 'color': 'red', 'radius': 25})

Display the widgets.

In [ ]:
display(ipyw.HBox([ipyw.VBox([slider, w]), w.line_out]))

Stop marking when done.

In [ ]:
w.stop_marking()

Close the file pointer, if open.

In [ ]:
if pf is not None:
    pf.close()